<a href="https://colab.research.google.com/github/Abu-Musa38/colab/blob/main/Fish_Health_database_canada_project_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
df=pd.read_csv('/content/Fish_Health_Database.csv', encoding='latin1',header=0)
df.head()

,CASE /,START DATE (yyyy-mm-dd) /,LOCATION NAME /,LATITUDE (Decimal Degrees) /,LONGITUDE (Decimal Degrees) /,SPECIES NAME /,LIFE STAGE DESCRIPTION /,ETIOLOGICAL DESCRIPTION /
0,CAS,DATE DE DÉBUT (aaaa-mm-jj),NOM DU LIEU,LATITUDE (Degrés Décimaux),LONGITUDE (Degrés Décimaux),NOM DE L'ESPÈCE,DESCRIPTION DU STADE BIOLOGIQUE,DESCRIPTION ÉTIOLOGIQUE
1,980261,NaN,NADINA RIVER,54.000859,-126.532768,SOCKEYE SALMON,SPENT ADULT,DERMOCYSTIDIUM
2,980261,NaN,NADINA RIVER,54.000859,-126.532768,SOCKEYE SALMON,SPENT ADULT,ICHTHYOPHTHIRIUS SP.
3,980258,NaN,FULTON RIVER HATCHERY,54.853338,-126.475918,SOCKEYE SALMON,FRY (0 - 4 MONTHS),PARASITES NOT DETECTED
4,580002,1958-12-04,FORT BABINE HATCHERY,55.044622,-126.312091,CHINOOK SALMON,YEARLING,CYATHOCEPHALUS


In [ ]:
# Current columns
df.columns

Index(['CASE /  ', 'START DATE (yyyy-mm-dd) /', 'LOCATION NAME /',
       'LATITUDE (Decimal Degrees) /', 'LONGITUDE (Decimal Degrees) /',
       'SPECIES NAME /', 'LIFE STAGE DESCRIPTION /',
       'ETIOLOGICAL DESCRIPTION /'],
      dtype='object')

In [ ]:
df.columns = [
    'Case_ID',
    'Start_Date',
    'Location_Name',
    'Latitude',
    'Longitude',
    'Species_Name',
    'Life_Stage_Description',
    'Etiological_Description'
    ]


In [ ]:
df.head()

,Case_ID,Start_Date,Location_Name,Latitude,Longitude,Species_Name,Life_Stage_Description,Etiological_Description
0,CAS,DATE DE DÉBUT (aaaa-mm-jj),NOM DU LIEU,LATITUDE (Degrés Décimaux),LONGITUDE (Degrés Décimaux),NOM DE L'ESPÈCE,DESCRIPTION DU STADE BIOLOGIQUE,DESCRIPTION ÉTIOLOGIQUE
1,980261,NaN,NADINA RIVER,54.000859,-126.532768,SOCKEYE SALMON,SPENT ADULT,DERMOCYSTIDIUM
2,980261,NaN,NADINA RIVER,54.000859,-126.532768,SOCKEYE SALMON,SPENT ADULT,ICHTHYOPHTHIRIUS SP.
3,980258,NaN,FULTON RIVER HATCHERY,54.853338,-126.475918,SOCKEYE SALMON,FRY (0 - 4 MONTHS),PARASITES NOT DETECTED
4,580002,1958-12-04,FORT BABINE HATCHERY,55.044622,-126.312091,CHINOOK SALMON,YEARLING,CYATHOCEPHALUS


In [ ]:
df.isnull().sum()

,0
Case_ID,0
Start_Date,3
Location_Name,0
Latitude,418
Longitude,418
Species_Name,0
Life_Stage_Description,0
Etiological_Description,0


In [ ]:
df = df.dropna(subset=['Latitude', 'Longitude'])

In [ ]:
df = df.drop(columns=['Case_ID', 'Start_Date'])

In [ ]:
df.isnull().sum()

,0
Location_Name,0
Latitude,0
Longitude,0
Species_Name,0
Life_Stage_Description,0
Etiological_Description,0


In [ ]:
df.shape

(6638, 6)

In [ ]:

df['Etiological_Description'] = df['Etiological_Description'].str.upper()

unhealthy_keywords = ['PARASITE', 'BACTERIA', 'VIRUS', 'FUNGUS', 'DEFICIENCY']

df['Health'] = df['Etiological_Description'].apply(
    lambda x: 'Unhealthy' if any(word in x for word in unhealthy_keywords) else 'Healthy'
)

print(df['Health'].value_counts())


Health
Healthy      5473
Unhealthy    1165
Name: count, dtype: int64


In [ ]:
df.replace({'Health': {'Healthy': 0, 'Unhealthy': 1}}, inplace=True)

/tmp/ipython-input-48-1353733829.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Health': {'Healthy': 0, 'Unhealthy': 1}}, inplace=True)


In [ ]:
if df.iloc[0]['Latitude'] == 'LATITUDE (Degrés Décimaux)':
    df = df.drop(index=0).reset_index(drop=True)

In [ ]:
df.head()

,Location_Name,Latitude,Longitude,Species_Name,Life_Stage_Description,Health
0,85,54.000859,-126.532768,16,18,0
1,43,54.853338,-126.475918,16,9,1
2,40,55.044622,-126.312091,1,21,0
3,122,49.457643,-124.816229,16,21,1
4,122,49.457643,-124.816229,16,21,1


In [ ]:
le = LabelEncoder()
df['Location_Name'] = le.fit_transform(df['Location_Name'])
df['Species_Name'] = le.fit_transform(df['Species_Name'])
df['Life_Stage_Description'] = le.fit_transform(df['Life_Stage_Description'])


In [ ]:
df.head()

,Location_Name,Latitude,Longitude,Species_Name,Life_Stage_Description,Health
0,85,54.000859,-126.532768,15,17,0
1,43,54.853338,-126.475918,15,8,1
2,40,55.044622,-126.312091,1,20,0
3,121,49.457643,-124.816229,15,20,1
4,121,49.457643,-124.816229,15,20,1


In [ ]:
df.drop(columns='Etiological_Description', inplace=True)

KeyError: "['Etiological_Description'] not found in axis"

In [ ]:
df.head()

,Location_Name,Latitude,Longitude,Species_Name,Life_Stage_Description,Health
0,85,54.000859,-126.532768,15,17,0
1,43,54.853338,-126.475918,15,8,1
2,40,55.044622,-126.312091,1,20,0
3,121,49.457643,-124.816229,15,20,1
4,121,49.457643,-124.816229,15,20,1


In [ ]:
X = df.drop(columns='Health', axis=1)
Y = df['Health']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
pip install xgboost


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=2)

# Train model
model.fit(X_train, Y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [19:04:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
df = df.drop(index=0)  # drop first row
df.reset_index(drop=True, inplace=True)  # reset index


In [ ]:
df.head()

,Location_Name,Latitude,Longitude,Species_Name,Life_Stage_Description,Health
0,121,49.457643,-124.816229,15,20,1
1,121,49.457643,-124.816229,15,20,1
2,121,49.457643,-124.816229,15,20,0
3,121,49.457643,-124.816229,15,0,0
4,8,49.399036,-124.616359,2,8,0


In [ ]:
# If you already loaded df, clean it:
df = df.drop(index=0)
df.reset_index(drop=True, inplace=True)

# Convert to numeric
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Drop rows with missing Lat/Lon
df = df.dropna(subset=['Latitude', 'Longitude'])

# Now prepare X and Y
X = df[['Latitude', 'Longitude', 'Species_Name', 'Life_Stage_Description']]
Y = df['Health']

# Make sure other columns are numeric too, convert if needed
X = X.apply(pd.to_numeric, errors='coerce')
X = X.dropna()

# Split data, train your model etc.


In [ ]:
# Step 10: Accuracy Check
X_train_prediction = model.predict(X_train)
train_accuracy = accuracy_score(Y_train, X_train_prediction)

X_test_prediction = model.predict(X_test)
test_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.844197437829691
Test Accuracy: 0.8162650602409639


[0]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
